In [2]:
# Dataset: Cell Body Segmentation
# Source: Hill, LaPan, Li and Haney (2007). Impact of image segmentation 
# on high-content screening data quality for SK-BR-3 cells, BMC 
# Bioinformatics, Vol. 8, pg. 340
# URL: https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-8-340
# Description: Hill, LaPan, Li and Haney (2007) develop models to predict 
# which cells in a high content screen were well segmented. The data 
# consists of 119 imaging measurements on 2019. 
# The original analysis used 1009 for training and 1010 as a test set. 
# The outcome class is contained in a factor variable called Class with 
# levels "PS" for poorly segmented and "WS" for well segmented.

#Step 0
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
#Step 1: Load Data
